In [1]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings# warning filter
import scipy as sp #pivot egineering


#ML model
from sklearn.metrics.pairwise import cosine_similarity


#default theme and settings
pd.options.display.max_columns

#warning hadle
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [2]:
for dirname, _, filenames in os.walk('C:\\Users\\aadri\\OneDrive\\Desktop\\animeRec'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\aadri\OneDrive\Desktop\animeRec\anime.csv
C:\Users\aadri\OneDrive\Desktop\animeRec\rating.csv


In [3]:
rating_path = "C:\\Users\\aadri\\OneDrive\\Desktop\\animeRec\\rating.csv"
anime_path = "C:\\Users\\aadri\\OneDrive\\Desktop\\animeRec\\anime.csv"

In [4]:
rating_df = pd.read_csv(rating_path)
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
anime_df = pd.read_csv(anime_path)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
print(f"anime set (row, col): {anime_df.shape}\n\nrating set (row, col): {rating_df.shape}")

anime set (row, col): (12294, 7)

rating set (row, col): (7813737, 3)


In [7]:
print("Anime:\n")
print(anime_df.info())
print("\n","*"*50,"\nRating:\n")
print(rating_df.info())

Anime:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None

 ************************************************** 
Rating:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


In [8]:
print("Anime missing values (%):\n")
print(round(anime_df.isnull().sum().sort_values(ascending=False)/len(anime_df.index),4)*100) 
print("\n","*"*50,"\n\nRating missing values (%):\n")
print(round(rating_df.isnull().sum().sort_values(ascending=False)/len(rating_df.index),4)*100)

Anime missing values (%):

rating      1.87
genre       0.50
type        0.20
anime_id    0.00
name        0.00
episodes    0.00
members     0.00
dtype: float64

 ************************************************** 

Rating missing values (%):

user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [9]:
print(anime_df['type'].mode())
print(anime_df['genre'].mode())

0    TV
Name: type, dtype: object
0    Hentai
Name: genre, dtype: object


In [10]:
# deleting anime with 0 rating
anime_df=anime_df[~np.isnan(anime_df["rating"])]

# filling mode value for genre and type
anime_df['genre'] = anime_df['genre'].fillna(
anime_df['genre'].dropna().mode().values[0])

anime_df['type'] = anime_df['type'].fillna(
anime_df['type'].dropna().mode().values[0])

#checking if all null values are filled
anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [11]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x==-1 else x)
rating_df.head(20)

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
5,1,355,NaN
6,1,356,NaN
7,1,442,NaN
8,1,487,NaN
9,1,846,NaN


In [12]:
#step 1
anime_df = anime_df[anime_df['type']=='TV']

#step 2
rated_anime = rating_df.merge(anime_df, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])

#step 3
rated_anime =rated_anime[['user_id', 'name', 'rating']]

#step 4
rated_anime_7500= rated_anime[rated_anime.user_id <= 7500]
rated_anime_7500.head()

,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81


In [13]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [14]:
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# step 2
pivot_n.fillna(0, inplace=True)

# step 3
pivot_n = pivot_n.T

# step 4
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

# step 5
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

In [15]:
#model based on anime similarity
anime_similarity = cosine_similarity(piv_sparse)

#Df of anime similarities
ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [16]:
   def anime_recommendation(ani_name):
    number=1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1  

In [17]:
anime_recommendation('Fairy Tail')

Recommended because you watched Fairy Tail:

#1: Shingeki no Kyojin, 40.86% match
#2: No Game No Life, 40.36% match
#3: Log Horizon, 39.63% match
#4: Magi: The Labyrinth of Magic, 39.13% match
#5: Noragami, 38.99% match


In [18]:
def top_anime(value):
    max_rating = anime_df[anime_df['genre'].apply(lambda x: value in x)]['rating'].max()
    return max_rating

In [25]:
top_anime('Action')

9.26

In [27]:
#def find_top_5_names(dropdown.get()):
    #df = pd.read_csv(csv_file)
 #   filtered_df = anime_df[anime_df['genre'].apply(lambda x: value in x)]
#    sorted_df = filtered_df.sort_values(by='rating', ascending=False)
 #   top_5_names = sorted_df['name'].head(5).tolist()
  #  return top_5_names

In [29]:
#find_top_5_names('Horror')

In [30]:
def gener():
    all_gener = set()
    for gener_list in anime_df['genre']:
        all_gener.update(gener_list)
    print(f'All unique values of gener: {all_gener}')

In [31]:
anime_df.genre

1        Action, Adventure, Drama, Fantasy, Magic, Mili...
2        Action, Comedy, Historical, Parody, Samurai, S...
3                                         Sci-Fi, Thriller
4        Action, Comedy, Historical, Parody, Samurai, S...
5                   Comedy, Drama, School, Shounen, Sports
                               ...                        
10879                                           Cars, Kids
10882    Action, Adventure, Comedy, Demons, Magic, Supe...
10889                                                 Kids
10892               Adventure, Comedy, Historical, Romance
10895                                               Comedy
Name: genre, Length: 3671, dtype: object

In [5]:
import tkinter as tk
from tkinter import ttk

def find_top_5_names(event):
    #df = pd.read_csv(csv_file)
    var=dropdown_var.get()
    filtered_df = anime_df[anime_df['genre'].apply(lambda x: var in x)]
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)
    top_5_names = sorted_df['name'].head(5).tolist()
    output_text.delete(1.0, tk.END)  # Clear the text box
    for item in top_5_names:
        output_text.insert(tk.END, f"{item}\n")






# Create the main window
root = tk.Tk()
root.title("Dropdown and Textbox Example")

# Create a frame to hold the dropdown menu
dropdown_frame = ttk.Frame(root)
dropdown_frame.pack(side=tk.LEFT, padx=10, pady=10)

# Create a label for the dropdown menu
dropdown_label = ttk.Label(dropdown_frame, text="Select an option:")
dropdown_label.pack(padx=10, pady=5)

# Create a variable to store the selected option
dropdown_var = tk.StringVar()

# Create the dropdown menu

dropdown = ttk.Combobox(dropdown_frame, textvariable=dropdown_var, values=options)
dropdown.pack(padx=10, pady=5)

# Bind a function to the dropdown selection event
dropdown.bind("<<ComboboxSelected>>",find_top_5_names)

# Create a frame to hold the text box
textbox_frame = ttk.Frame(root)
textbox_frame.pack(side=tk.RIGHT, padx=10, pady=10)

# Create a label for the text box
textbox_label = ttk.Label(textbox_frame, text="Output:")
textbox_label.pack(padx=10, pady=5)

# Create a text box for output
output_text = tk.Text(textbox_frame, width=50, height=30)
output_text.pack(padx=10, pady=5)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\aadri\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\aadri\AppData\Local\Temp\ipykernel_34224\641361596.py", line 7, in find_top_5_names
    filtered_df = anime_df[anime_df['genre'].apply(lambda x: var in x)]
NameError: name 'anime_df' is not defined


In [ ]:
import tkinter as tk
from tkinter import ttk

def update_second_textbox():
    value = input_text.get()
    second_textbox.delete(1.0, tk.END)  # Clear the second textbox
    second_textbox.insert(tk.END, f"Value entered: {value}")

# Create the main window
root = tk.Tk()
root.title("Two Segments Example")

# Create the first segment (left side)
first_segment = ttk.Frame(root)
first_segment.pack(side=tk.LEFT, padx=10, pady=10, expand=True, fill="both")

# Create a label for the first segment
label1 = ttk.Label(first_segment, text="Dropdown:")
label1.pack(pady=5)

# Create a dropdown box for the first segment
options = ["Option 1", "Option 2", "Option 3"]
dropdown = ttk.Combobox(first_segment, values=options)
dropdown.pack()

# Create a textbox to store the answer of the dropdown in the first segment
output_text = tk.Text(first_segment, height=5, width=30)
output_text.pack(pady=5)

# Create the second segment (right side)
second_segment = ttk.Frame(root)
second_segment.pack(side=tk.LEFT, padx=10, pady=10, expand=True, fill="both")

# Create a label for the second segment
label2 = ttk.Label(second_segment, text="Enter Value:")
label2.pack(pady=5)

# Create a textbox to enter a value in the second segment
input_text = ttk.Entry(second_segment)
input_text.pack()

# Create a second textbox to show the value entered in the first textbox
second_textbox = tk.Text(second_segment, height=5, width=30)
second_textbox.pack(pady=5)

# Create a button to update the second textbox
update_button = ttk.Button(second_segment, text="Update", command=update_second_textbox)
update_button.pack(pady=5)

root.mainloop()
